# Visión por Computador - Práctica 2
## Detección de puntos relevantes y Construcción de panoramas

### 1. Detección de puntos Harris multiescala (MOPS, Brown, Szeliski, Winder, 2004). 

#### En este apartado se trata de implementar el detector de regiones descrito en el artículo mencionado. En este algoritmo se suponen que todas las imágenes están tomadas en una escala semejante (zoom y distancia iguales) por lo que el objetivo es extraer todas las regiones relevantes que haya a distintas escalas enteras de las imágenes. Por cada región detectada necesitaremos guardar la siguiente información: (las coordenadas x ,y de su centro, su orientación, su escala)). Usar para ello la estructura que resulte más fácil. Presentar los resultados con las imágenes Yosemite.rar.

##### a) Escribir una función que extraiga la lista potencial de puntos Harris en una imagen a distintas escalas. Para ello construiremos una pirámide Gaussiana de la imagen con 3 escalas usando $\sigma = 1$. Sobre cada nivel de la pirámide usar la función OpenCV cornerEigenValsAndVecs para extraer los mapas de auto-valores de la matriz Harris en cada píxel (( fijar los valores de blockSize en el rango [3-13] y ksize en el rango [3-9]) , usar la versión de nivel de gris de las imágenes). Crear una matriz con el valor del criterio selección Harris asociado a cada píxel usando ($k=0.04$). Implementar la fase de supresión de valores non-máximos sobre dicha matriz. Ordenar de mayor-a-menor en cada escala los puntos resultantes de acuerdo a su valor Harris. Seleccionar al menos 1500 puntos de entre los de mayor valor distribuidos entre las distintas escalas (las escalas más bajas tendrán más puntos: 70-20-10). Mostrar el resultado dibujando un círculo sobre la imagen original de radio proporcional a la escala y centro las coordenadas de los puntos.

##### b) Extraer los valores (x,y, escala) de los puntos resultantes en el apartado anterior en un vector y refinar su posición espacial a nivel sub-píxel usando la función OpenCV usando cornerSubPix()) con la imagen del nivel de pirámide correspondiente. Actualizar los datos (x,y ,escala) de cada uno de los puntos encontrados. 

##### c)  Calcular la orientación relevante de cada punto Harris, usando un alisamiento fuerte de las derivadas en x e y de las imágenes, en la escala correspondiente, como propone el paper MOPS de Brown&Szeliski&Winder. (Apartado 2.5) y añadir la información del ángulo al vector de información del punto. Pintar sobre la imagen de círculos anterior un radio en cada círculo indicando la orientación estimada en ese punto.

- **Apartado a**: para resolver este ejercicio, se han desarrollado varias funciones. Iremos explicándolas poco a poco y viendo su desarrollo.
    
    Antes de realizar la pirámide Gaussiana, lo primero es ver si la imagen es divisible entre el número de divisiones que vamos a hacer. Esto se hace para que, una vez encontrado un punto relevante en la escala $S_i$, sea más sencillo recuperar la posición original en la que se encontraba el píxel, tan solo multiplicando tantas veces como se haya reducido la imagen para encontrar ese punto. De esto se encarga la función ```prepare_img_to_harris_points``` que recibe como parámetro la imagen de la que detectaremos los puntos.
    
    Esta función, recibe la imagen, la pasa a escala de grises y añade tantos píxeles como sean necesarios para poder tener una imagen cuyos puntos Harris sean fáciles de recuperar a la escala original.

In [2]:
import math
import cv2
import random
import numpy as np
from functions import *

def prepare_img_to_harris_points(img):
    # Extraemos las dimensiones de la imagen, para que,
    # en caso de que sean de tamaño impar, añadirle las filas
    # o columnas que correspondan, para que, al reducir,
    # podamos recuperar fácilmente las coordenadas de los puntos
    # harris.
    alt, anch = img.shape[:2]

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    aux=[]
    # Ensanchamos una fila de la imagen
    if alt % 4 != 0 and anch % 4 == 0:
        aux = np.ones(shape=(alt + alt % 4, anch), dtype=np.uint8)
        insert_img_into_other(img_src=gray_img, img_dest=aux, pixel_left_top_col=0,
                              pixel_left_top_row=0, substitute=True)

    # Ensanchamos una columna de la imagen
    elif alt % 4 == 0 and anch % 4 != 0:
        aux = np.ones(shape=(alt, anch + anch % 4), dtype=np.uint8)
        insert_img_into_other(img_src=gray_img, img_dest=aux, pixel_left_top_col=0,
                              pixel_left_top_row=0, substitute=True)

    # Ensanchamos una fila y una columna de la imagen
    elif alt % 4 != 0 and anch % 4 != 0:
        aux = np.ones(shape=(alt + alt % 4, anch + anch % 4), dtype=np.uint8)
        insert_img_into_other(img_src=gray_img, img_dest=aux, pixel_left_top_col=0,
                              pixel_left_top_row=0, substitute=True)
    # se queda igual que la original
    else:
        aux = np.copy(gray_img)

    return aux


  Lo siguiente, es generar la pirámide Gaussiana con la función ```generate_gaussian_pyramide``` definida en la práctica anterior, y obtener los puntos en los que su valor de Harris supere un umbral mínimo. En este caso, el umbral se ha definido con un mínimo de $10^{-6}$ para que haya un gran número de puntos para poder obtener suficientes puntos de calidad.
  
  Para obtener estos puntos, se ha implementado la función ```get_eigenVals_and_eigenVecs```, que recibe como parámetros la pirámide de imágenes de la que se extraerán los puntos, el umbral, el tamaño de bloque (que es el tamaño del vecindario del punto) y el ksize (parámetro para el operador de Sobel). La función tiene un bucle en el que se llama a la función de OpenCV ```cornerEigenValsAndVecs``` que nos devuelve los valores de $\lambda_1$, $\lambda_2$, los autovalores y autovectores de ambos. Con ellos calcularemos la traza y el determinante para cada una de las imágenes para obtener el valor de Harris de cada uno de los píxeles de la función. Este valor lo calcularemos siguiendo el criterio histórico de Harris: $$ f = determinant(M) - \alpha \cdot trace(M)^2 $$ donde $\alpha$ es una constante que usaremos con valor 0.04, representada en el código como $k$. Siguiendo esto, obtendremos los valores de cada punto en la matriz, y almacenaremos los índices de los puntos que superan el umbral. Esto lo repetiremos para cada escala:

In [1]:
# Criterio histórico de Harris
harrisCriterio = lambda det, tr, k=0.04: det - k*(tr**2)

# Obtiene los valores de harris y los indices
# de los puntos que superan el umbral
def get_eigenVals_and_eigenVecs(pyramide, thresdhold, blockS, kSize):
    eingen_vals_and_vecs = []
    strong_values = []

    for im in pyramide:
        # Obtenemos la matriz de con los autovalores de la matriz
        # y los respectivos autovectores para cada uno de los autovalores
        result = cv2.split(cv2.cornerEigenValsAndVecs(src=im.astype(np.uint8),
                                                      blockSize=blockS, ksize=kSize))
        # Calculamos el determinante como el producto de los autovalores
        det = cv2.mulSpectrums(result[0], result[1], flags=cv2.DFT_ROWS)
        # Calculamos la traza como la suma de los autovalores
        trace = result[0] + result[1]
        # Realizamos la función de valoración de Harris
        eingen_vals_and_vecs.append(harrisCriterio(det, trace))
        # Y obtenemos los índices de los píxeles que sobrepasan el umbral mínimo
        strong_values.append(np.where(eingen_vals_and_vecs[-1] > thresdhold))

    return eingen_vals_and_vecs, strong_values

Tras esto, pasaremos a obtener los puntos que conforman un máximo local dentro de su entorno, en los puntos obtenidos en la función anterior. Para ello, definiremos una serie de funciones que se encargarán de hacer este trabajo. Estas funciones son:
- ```local_maximum```: recibe como parámetro un entorno o vecindario de un punto perteneciente a la matriz con los valores de Harris obtenidos en la función anterior, y comprueba si el valor del centro, es el máximo local.
- ```get_local_maximun```: se encarga de obtener todos los máximos locales de los puntos extraídos, para obtener los puntos de mayor calidad de los que han sobrepasado el umbral. Tras esto, devolvemos las coordenadas de los puntos máximos, junto con su valor de Harris.

In [2]:
# Esta función comprueba si el centro del entorno es
# un máximo local o no
def local_maximun(environment):
    floor = math.floor
    height, width = environment.shape[:2]
    center = environment[floor(width/2),floor(height/2)]
    return center == np.max(environment)


def get_local_maximun(imgs, index_mask, mask_size):
    # obtenemos los índices de los puntos que han sobrepasado
    # el umbral mínimo
    escala = 1
    img = 0
    xy_bests_points = []
    harrisV_bests_points = []
    for i in index_mask:
        # Arrays para almacenar las coordenadas en X y en Y
        coord_x = []
        coord_y = []
        rows = i[0]
        cols = i[1]
        # Extendemos la imagen para poder captar fácilmente
        # los máximos locales de los bordes
        imgaux = extend_image_n_pixels(img_src=imgs[img],
                                       border_type=4,
                                       n_pixels=mask_size)
        # Array para almacenar los máximos locales almacenados
        maxHs = []
        for k in range(len(rows)):
            # Obtenemos las cuatro esquinas de la región a analizar
            left = rows[k]
            right = (rows[k]+mask_size)
            top = cols[k]
            down = (cols[k] + mask_size)
            # Y comprobamos si la región contiene en su centro un máximo local
            if local_maximun(imgaux[left:right, top:down]):
                # si lo es, almacenamos su posición y su valor harris
                coord_x.append(rows[k])
                coord_y.append(cols[k])
                maxHs.append(imgs[img][rows[k],cols[k]])

        # Insertamos los puntos máximos y su valor en la lista
        xy_bests_points.append(np.array([np.array(coord_x), np.array(coord_y)]))
        # xy_bests_points.append(maxls_xy)
        harrisV_bests_points.append(maxHs)
        img += 1
        escala *= 2

    return xy_bests_points, harrisV_bests_points


Una vez encontrados los máximos locales, pasamos a encontrar y seleccionar los mejores puntos de cada escala. En la escala más baja (la original) encontraremos el mayor número de puntos, y cada vez que subamos un nivel en la pirámide, seleccionaremos menos, escogiendo el 70%, el 20% y el 10% en cada uno de los niveles respectivamente.

De esto se encarga la función ```get_best_points```, que en cada nivel, ordena los puntos máximos de mayor a menor según su valor de Harris, y selecciona el porcentaje de puntos que queremos de cada nivel.

In [2]:
def get_best_points(img_points, xy_points, harrisV, n_points):
    # Pasamos a poner a 1 los puntos con máximos locales
    it = 0
    floor = math.floor
    # Esto representa el porcentaje de puntos que tomaremos de cada escala
    # tomando del primer nivel el 70%, del segundo el 20% y del último el 10%
    percentages = [.7, .2, .1]
    escala = 1
    selected_points = []
    # Empezamos a recorrer los puntos que hemos extraído como máximos locales
    for points in harrisV:
        # ordenamos los puntos. Como argsort los da ordenados
        # de menor a mayor, invertimos el vector para obtenerlos
        # de mayor a menor.
        index = np.argsort(points)[::-1]
        # tomamos las coordenadas del % de puntos mejores
        coord_xy = [xy_points[it][0][index[0:floor(n_points * percentages[it])]],
                    xy_points[it][1][index[0:floor(n_points * percentages[it])]]]

        # Almacenamos los mejores puntos de cada escala
        # ya ordenados, y sin
        selected_points.append(np.array(coord_xy).T)

        # Almacenamos los puntos en una lista para poder
        # dibujar los círculos
        coordinates_for_circles = [xy_points[it][1][index[0:floor(n_points * percentages[it])]] * escala,
                                        xy_points[it][0][index[0:floor(n_points * percentages[it])]] * escala,]
        # coordinates_for_circles.append(coord_xy*escala)
        # y los ponemos a 1
        img_points[coordinates_for_circles[::-1]] = 255
        it += 1
        escala *= 2

    show_img(img_points, "Primeros puntos seleccionados")

    return selected_points

* **Apartado b**: una vez que tenemos los mejores puntos de cada nivel, pasamos a refinar estos puntos a nivel de subpíxel. Para ello, implementaremos la función ```refine_points```, que recibe como parámetros la pirámide de imágenes, y los puntos seleccionados. 
    Para cada punto de cada nivel, se refina a nivel de subpíxel llamando a la función de *OpenCV* ```cornerSubPix```, almacenandolo en una lista multinivel los puntos que hemos refinado.

In [1]:
def refine_points(pyramide, selected_points):
    refined_points = []
    it = 0

    for img in pyramide:
        float_esquinas = np.array(selected_points[it], dtype=np.float32).copy()
        cv2.cornerSubPix(image=img.astype(np.float32), corners=float_esquinas,
                         winSize=(5, 5), zeroZone=(-1, -1),
                         criteria=(cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_COUNT, 10, 0.01))
        it += 1
        refined_points.append(float_esquinas)

    return refined_points

* **Apartado c**: una vez refinados los píxeles a nivel de subpíxel, pasamos a obtener la dirección estimada del gradiente en ese punto. Para ello, derivamos la imagen en $\mathcal{X}$ y en $\mathcal{Y}$. Una vez derivada la imagen, para obtener el ángulo, realizamos lo siguiente: $$\arctan\left(\frac{dy}{dx}\right)$$

    Una vez hecho esto, el ángulo tendremos que convertirlo de radianes a grados para trabajar con OpenCV, multiplicando por 180 y dividiendo por $\pi$. Tras esto almacenamos los puntos en una lista.

In [1]:
def detect_angles(pyramide, refined_points):
    # inicializamos la lista con los ángulos
    angles = []
    # y recorremos las tres escalas
    for scale in range(3):
        dx_img, dy_img = get_derivates_of(img=pyramide[scale], sigma=4.5)
        # Obtenemos los indices para poder
        indices = np.array(refined_points[scale].T, dtype=int)
        # Calculamos los ángulos de forma vectorizada, donde
        # el ángulo es:
        #           ang = atan( dy/dx )
        angles.append((np.arctan2(dy_img[indices[0], indices[1]],
                                  dx_img[indices[0], indices[1]]))*180/np.pi)

    # Devolvemos los ángulos
    return angles

Tras esto, solo nos queda presentar si se desea el resultado obtenido, y devolverlo. Para presentarlo, se ha definido la siguiente función, ```show_result```, que recibe como parámetros la imagen original, los puntos refinados y el ángulo. Para cada uno de las escalas, se usa un radio distinto, donde para los niveles más altos de la pirámide, se usan radios más grandes. A su vez, cambiará el color según las escalas, siendo de color azul la escala original, verde la intermedia y roja la última escala.

In [4]:
def show_result(img, refined_points, angles, thick):
    aux2 = np.copy(img)
    floor = math.floor
    sin = np.sin
    cos = np.cos
    radio = [5, 10, 20]
    colors = [(175, 0, 0), (0, 175, 0), (0, 0, 175)]
    size = 1
    for scale in range(3):
        for i in random.sample(range(len(refined_points[scale])), 100):
        # for i in range(len(refined_points[scale])):
            punto = refined_points[scale][i].astype(np.int) * size
            angle = angles[scale][i]
            cv2.circle(img=aux2, center=(punto[1], punto[0]),
                       radius=radio[scale], color=colors[scale],
                       thickness=thick)
            cv2.arrowedLine(img=aux2, pt1=(punto[1], punto[0]),
                            pt2=(punto[1] + floor(sin(angle) * radio[scale]),
                                 punto[0] + floor(cos(angle) * radio[scale])),
                            color=colors[scale],thickness=thick)
        size *= 2

    show_img(aux2, 'Puntos y ángulos')

Con todo esto, podemos generar la función ```extract_harris_points``` que recibe como parámetros la imagen de entrada, blockS, kSize, el umbral, el número de puntos que queremos extraer y parámetros para si queremos mostrar los resultados o no, y el grosor de la línea que forma la circunferencia que indica la posición y escala a la que se ha encontrado el punto, y el radio que indica su dirección aproximada.

In [ ]:
def extract_harris_points(img, blockS, kSize, thresdhold, n_points = 1500,
                          show_best_points = True,
                          thick = 1):
    #######################################
    # Apartado a: extrare lista potencial
    # de puntos Harris
    #######################################
    alt, anch = img.shape[:2]
    aux = prepare_img_to_harris_points(img)
    # obtenemos la pirámide gaussiana
    pyramide = generate_gaussian_pyramide(img_src=aux, subsample_factor=2, n_levels=3)

    # Obtenemos los autovalores y autovectores, junto
    # con los puntos que superan el umbral
    eingen_vals_and_vecs, strong_values = \
        get_eigenVals_and_eigenVecs(pyramide, thresdhold, blockS, kSize)

    # pasamos a eliminar los no máximos
    xy_points, harrisV = get_local_maximun(imgs=eingen_vals_and_vecs,
                                           index_mask=strong_values,
                                           mask_size=3)
    # inicializamos una imagen binaria (0,255) para
    # representar los máximos locales de la imagen
    img_points = np.zeros(shape=img.shape,dtype=np.uint8)
    # Obtenemos los mejores puntos para cada uno de
    # los niveles

    selected_points = get_best_points(img_points, xy_points, harrisV, n_points)

    #######################################
    # Apartado b, refinar las coordenadas
    #######################################
    refined_points = refine_points(pyramide, selected_points)

    ####################################
    # Apartado c, detectar orientacion
    ####################################
    # Eliminamos aquellos puntos que puedan haberse excedido del
    # tamaño de la imagen
    refined_points[0] = np.delete(refined_points[0],
                                  np.where(refined_points[0][:, 0] > alt), 0)
    refined_points[0] = np.delete(refined_points[0],
                                  np.where(refined_points[0][0, :] > anch), 0)

    # Obtenemos los ángulos de los gradientes
    angles =  detect_angles(pyramide, refined_points)
    # y mostramos el resultado si procede
    if show_best_points:
        show_result(img, refined_points, angles, thick)

    return refined_points, angles